# Dataset improvements

In [2]:
!pip install pandas_ta

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas_ta-0.3.14b0-py3-none-any.whl


In [3]:
import pandas as pd

In [4]:
# read data from gh
df = pd.read_csv('https://raw.githubusercontent.com/xsa-dev/Trading-Bot---Deep-Reinforcement-Learning/master/Bot_code_and_models/input/aggregated_dataset.csv')
df

,Open,High,Low,Close,Volume
0,18469.46,18496.65,18469.29,18491.30,556.77573
1,18491.30,18492.98,18449.00,18467.38,615.91775
2,18468.82,18477.84,18450.00,18451.43,472.64174
3,18451.43,18472.36,18451.05,18455.32,373.01516
4,18455.32,18460.55,18440.00,18443.34,421.05380
...,...,...,...,...,...
525995,26763.88,26763.88,26760.00,26760.00,3.53700
525996,26760.01,26760.01,26760.00,26760.01,1.99524
525997,26760.00,26760.01,26760.00,26760.01,2.91796
525998,26760.01,26944.62,26760.00,26911.65,517.04627


## 1. Учитывать относительные значения: Вместо абсолютных значений свечей можно пересчитать их относительно скользящей средней, что может помочь в создании более надежных моделей.

In [5]:
# Вычислите скользящую среднюю (SMA) для столбца 'Close' с периодом 13
df['SMA_13'] = df['Close'].rolling(window=13).mean()

# Пересчитайте значения 'Open', 'High', 'Low', 'Close' относительно SMA
##  2. Включить данные о скользящей средней: Добавление скользящей средней с периодом 13 в датасет может быть полезным для прогнозирования.
df['Open_SMA_13'] = df['Open'] / df['SMA_13']
df['High_SMA_13'] = df['High'] / df['SMA_13']
df['Low_SMA_13'] = df['Low'] / df['SMA_13']
df['Close_SMA_13'] = df['Close'] / df['SMA_13']
# ## 3. Учесть процент от значений скользящей средней: Это может предоставить модели информацию о том, насколько текущее значение отличается от среднего.
# ## 4. Включить объемы: Добавление данных о объемах за последние 1024 свечи может быть важным фактором для моделей.
df['Volume_1024'] = df['Volume'].rolling(window=1024).sum()

## 5. Работа с разными тикерами: Создание универсального датасета, который можно использовать для разных тикеров, предоставляя моделям разнообразные данные.
...

## 6. Размер окна для предсказаний: Учесть необходимость кормить модели окном размером 1024 бара для предсказания.

## 7. Учет профита: Включение информации о профите и установление баллов за профит может быть полезным для оценки эффективности модели.

In [6]:
# Вычислите столбец "Target" (-1, 0, 1)
lookahead_period = 5  # Период, на который вы хотите прогнозировать (5 свечей вперед)
df['Close_Lookahead'] = df['Close'].shift(-lookahead_period)
df['Target'] = (df['Close_Lookahead'] - df['Close']).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Удалите временный столбец "Close_Lookahead"
df.drop('Close_Lookahead', axis=1, inplace=True)

## 9. Ускорение процесса: Учитывайте цены, сравнивая их с ценой +5 баров для определения профита и оценки эффективности.

## Эти улучшения позволят создать более информативный и адаптируемый датасет для обучения моделей прогнозирования.


In [7]:
df['Target'].unique()

array([-1,  1,  0])

In [8]:
df

,Open,High,Low,Close,Volume,SMA_13,Open_SMA_13,High_SMA_13,Low_SMA_13,Close_SMA_13,Volume_1024,Target
0,18469.46,18496.65,18469.29,18491.30,556.77573,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,18491.30,18492.98,18449.00,18467.38,615.91775,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,18468.82,18477.84,18450.00,18451.43,472.64174,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,18451.43,18472.36,18451.05,18455.32,373.01516,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,18455.32,18460.55,18440.00,18443.34,421.05380,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
525995,26763.88,26763.88,26760.00,26760.00,3.53700,26769.045385,0.999807,0.999807,0.999662,0.999662,15083.94108,0
525996,26760.01,26760.01,26760.00,26760.01,1.99524,26768.392308,0.999687,0.999687,0.999686,0.999687,15068.56862,0
525997,26760.00,26760.01,26760.00,26760.01,2.91796,26767.408462,0.999723,0.999724,0.999723,0.999724,15064.60547,0
525998,26760.01,26944.62,26760.00,26911.65,517.04627,26778.088462,0.999325,1.006219,0.999325,1.004988,15574.29864,0
